This notebook trains a machine learning capable of detecting DeepFake (spoofed) voice samples and
distinguishing them from real human speech.

Based on PA – (Physical Access) type audio set given in ASVspoof 2019 Dataset.

Physical Access attacks happen when fake speech is played through a speaker and re-recorded using a microphone.Attacker plays a recording of your voice on a phone and System records it via microphone.These have characterstics Background noise, Device distortions, Echo and reverberation

In [1]:
import pandas as pd
import numpy as np
import os
import librosa
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Dataset selection & protocol analysis.

In [2]:
# organizing audio samples into training sets.
# creating a dataframe named training_protocols.

training_protocols_path = "/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVspoof2019_PA_cm_protocols/ASVspoof2019.PA.cm.train.trn.txt"

training_protocols = pd.read_csv(training_protocols_path, sep=" ", header=None)
training_protocols.columns = ["speaker", "file_id", "attack", "channel", "label"]
training_protocols["split"] = "train"

# Adding paths of Audio files to dataframe according to their file_id s.

AUDIO_BASE = "/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVspoof2019_PA_train/flac/"

training_protocols["audio_path"] = training_protocols["file_id"].apply(
    lambda x: f"{AUDIO_BASE}{x}.flac"
)

In [3]:
# Verifying audio paths exist.
training_protocols["file_exists"] = training_protocols["audio_path"].apply(
    lambda x: os.path.exists(x)
)
training_protocols["file_exists"].value_counts()

file_exists
True    54000
Name: count, dtype: int64

In [4]:
training_protocols.describe(include="all")

,speaker,file_id,attack,channel,label,split,audio_path,file_exists
count,54000,54000,54000,54000,54000,54000,54000,54000
unique,20,54000,27,10,2,1,54000,1
top,PA_0079,PA_T_0053984,aaa,AA,spoof,train,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,True
freq,2700,1,2000,7144,48600,54000,1,54000


In [5]:
training_protocols["label"].value_counts()

label
spoof       48600
bonafide     5400
Name: count, dtype: int64

In [6]:
training_protocols[["audio_path", "label", "split"]].head()

,audio_path,label,split
0,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train
1,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train
2,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train
3,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train
4,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train


In [7]:
# organizing audio samples into validation sets.
# creating a dataframe named development_protocols.

development_protocols_path = "/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVspoof2019_PA_cm_protocols/ASVspoof2019.PA.cm.dev.trl.txt"

development_protocols = pd.read_csv(development_protocols_path, sep=" ", header=None)
development_protocols.columns = ["speaker", "file_id", "attack", "channel", "label"]
development_protocols["split"] = "development"

# Adding paths of Audio files to dataframe according to their file_id s.

AUDIO_BASE = "/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVspoof2019_PA_dev/flac/"

development_protocols["audio_path"] = development_protocols["file_id"].apply(
    lambda x: f"{AUDIO_BASE}{x}.flac"
)

In [8]:
# Verifying audio paths exist.
development_protocols["file_exists"] = development_protocols["audio_path"].apply(
    lambda x: os.path.exists(x)
)
development_protocols["file_exists"].value_counts()

file_exists
True    29700
Name: count, dtype: int64

In [9]:
development_protocols.describe(include="all")

,speaker,file_id,attack,channel,label,split,audio_path,file_exists
count,29700,29700,29700,29700,29700,29700,29700,29700
unique,20,29700,27,10,2,1,29700,1
top,PA_0069,PA_D_0029700,aaa,-,spoof,development,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,True
freq,2700,1,1100,5400,24300,29700,1,29700


In [10]:
development_protocols["label"].value_counts()

label
spoof       24300
bonafide     5400
Name: count, dtype: int64

In [11]:
development_protocols[["audio_path", "label", "split"]].head()

,audio_path,label,split
0,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development
1,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development
2,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development
3,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development
4,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development


In [12]:
# organizing audio samples into testing sets.
# creating a dataframe named testing_protocols.

testing_protocols_path = "/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVspoof2019_PA_cm_protocols/ASVspoof2019.PA.cm.eval.trl.txt"

testing_protocols = pd.read_csv(testing_protocols_path, sep=" ", header=None)
testing_protocols.columns = ["speaker", "file_id", "attack", "channel", "label"]
testing_protocols["split"] = "test"

# Adding paths of Audio files to dataframe according to their file_id s.

AUDIO_BASE = "/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVspoof2019_PA_eval/flac/"

testing_protocols["audio_path"] = testing_protocols["file_id"].apply(
    lambda x: f"{AUDIO_BASE}{x}.flac"
)

In [13]:
# Verifying audio paths exist.
testing_protocols["file_exists"] = testing_protocols["audio_path"].apply(
    lambda x: os.path.exists(x)
)
testing_protocols["file_exists"].value_counts()

file_exists
True    134730
Name: count, dtype: int64

In [14]:
testing_protocols.describe(include="all")

,speaker,file_id,attack,channel,label,split,audio_path,file_exists
count,134730,134730,134730,134730,134730,134730,134730,134730
unique,67,134730,27,10,2,1,134730,1
top,PA_0016,PA_E_0137457,cbc,-,spoof,test,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,True
freq,2700,1,4990,18090,116640,134730,1,134730


In [15]:
testing_protocols["label"].value_counts()

label
spoof       116640
bonafide     18090
Name: count, dtype: int64

In [16]:
testing_protocols[["audio_path", "label", "split"]].head()

,audio_path,label,split
0,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,test
1,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,spoof,test
2,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,spoof,test
3,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,spoof,test
4,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,spoof,test


In [17]:
label_map = {"bonafide": 0, "spoof": 1}
training_protocols["label_enc"] = training_protocols["label"].map(label_map)
development_protocols["label_enc"] = development_protocols["label"].map(label_map)
testing_protocols["label_enc"] = testing_protocols["label"].map(label_map)

In [18]:
training_protocols[["audio_path", "label", "split", "label_enc"]].head()

,audio_path,label,split,label_enc
0,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train,0
1,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train,0
2,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train,0
3,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train,0
4,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,train,0


In [19]:
development_protocols[["audio_path", "label", "split", "label_enc"]].head()

,audio_path,label,split,label_enc
0,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development,0
1,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development,0
2,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development,0
3,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development,0
4,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,development,0


In [20]:
testing_protocols[["audio_path", "label", "split", "label_enc"]].head()

,audio_path,label,split,label_enc
0,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,bonafide,test,0
1,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,spoof,test,1
2,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,spoof,test,1
3,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,spoof,test,1
4,/kaggle/input/asvpoof-2019-dataset/PA/PA/ASVsp...,spoof,test,1


## Pipeline for Feature Extraction

In [21]:
# 1. Audio preprocessing pipeline.

def audio_preprocessing_pipeline(audio_path, target_sr=22050, target_duration=4.0):
                                     
    target_length = int(target_sr * target_duration)
    
    # Loading & resampling.
    audio, _ = librosa.load(audio_path, sr=target_sr)
    
    # Normalizing.
    audio = librosa.util.normalize(audio)
    
    # Trimming silence.
    audio, _ = librosa.effects.trim(audio)
    
    # Padding or cropping(if any).
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)))
    else:
        audio = audio[:target_length]
    
    return audio

# 2. Feature extraction pipeline to generate MFCC feature vectors.

def extract_mfcc_features(
    audio,
    sr=22050,
    n_mfcc=13
):    
    """
    Function Parameters:
        - audio:(output of preprocessing pipeline)
        - sr: sample rate (default 16kHz)
        - n_mfcc: number of MFCC coefficients
    """
    # Extracting MFCCs.
    mfccs = librosa.feature.mfcc(y=audio, n_mfcc=n_mfcc, sr=sr)
    
    # Extracting first / second MFCCs drivatives.
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    #print(mfccs.shape)
    #print(delta_mfccs.shape)
    #print(delta2_mfccs.shape)
    
    mfccs_features_combined = np.vstack([mfccs, delta_mfccs, delta2_mfccs])
    #print(mfccs_features_combined.shape)
    
    # Right now mfccs_features_combined.shape is (39, T), that's a 2d array having time as also a variable,
    # but classical ML models wants fixed lenght vectors or we as 1d arrays example: (78,).
    # For this we will take mean and standard deviation of mfccs_features_combined over duration of audio file (4 secs),
    # and pool them together into one vector of shape (78,) that we can feed into our model.
    
    mfcc_mean = np.mean(mfccs_features_combined, axis=1)
    mfcc_std = np.std(mfccs_features_combined, axis=1)
    
    # Hence MFCC feature vector is created.
    feature_vector = np.concatenate([mfcc_mean, mfcc_std])
    #print(feature_vector.shape)
    
    return feature_vector

## Pipeline for Feature Creation

In [22]:
def full_audio_processing(
    audio_path,
    target_sr=22050,
    target_duration=4.0,
    n_mfcc=13
):
    # Audio preprocessing.
    audio = audio_preprocessing_pipeline(
        audio_path=audio_path,
        target_sr=target_sr,
        target_duration=target_duration
    )

    # MFCC + Delta + Delta-Delta features extraction.
    feature_vector = extract_mfcc_features(
        audio=audio,
        sr=target_sr,
        n_mfcc=n_mfcc
    )

    return feature_vector

## Arranging Target and Predictors

In [23]:
def build_predictors_and_target_arrays(df):
    X = []
    y = []

    for _, row in df.iterrows():
        audio_path = row["audio_path"]
        label = row["label_enc"]

        features = full_audio_processing(audio_path)

        X.append(features)
        y.append(label)

    return np.array(X), np.array(y,)

In [ ]:
x_train, y_train = build_predictors_and_target_arrays(training_protocols)
x_dev, y_dev     = build_predictors_and_target_arrays(development_protocols)
x_test, y_test   = build_predictors_and_target_arrays(testing_protocols)

## Scaling Data

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_dev_scaled  = scaler.transform(x_dev)
x_test_scaled = scaler.transform(x_test)

In [ ]:
print(x_train.shape, y_train.shape)
print(x_dev.shape, y_dev.shape)
print(x_test.shape, y_test.shape)

## Classifier Using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=1000, class_weight="balanced",)

lr_model.fit(x_train_scaled, y_train)
# Model has been trained.

In [ ]:
# evaluating LR model on Development Dataset.
lr_y_dev_predictions = lr_model.predict(x_dev_scaled)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print("Dev Accuracy Score:")
print(accuracy_score(y_dev, lr_y_dev_predictions))

print("Dev Classification Report:")
print(classification_report(y_dev, lr_y_dev_predictions))

print("Dev Confusion Matrix:")
print(confusion_matrix(y_dev, lr_y_dev_predictions))

In [ ]:
lr_y_dev_probability = lr_model.predict_proba(x_dev_scaled)[:, 1]

from sklearn.metrics import roc_curve
import numpy as np

lr_fpr, lr_tpr, lr_thresholds = roc_curve(y_dev, lr_y_dev_probability)
lr_fnr = 1 - lr_tpr

lr_eer_idx = np.nanargmin(np.abs(lr_fnr - lr_fpr))
lr_eer = lr_fpr[lr_eer_idx]
lr_eer_threshold = lr_thresholds[lr_eer_idx]

print("Dev EER:", lr_eer)
print("EER Threshold:", lr_eer_threshold)

## Classifier Using Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svm_model = SVC(kernel="rbf", C=1.0, gamma="scale",class_weight="balanced",  probability=True)

svm_model.fit(x_train_scaled, y_train)
# Model has been trained.

In [ ]:
# evaluating SVM model on Development Dataset.
svm_y_dev_predictions = svm_model.predict(x_dev_scaled)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print("Dev Accuracy Score:")
print(accuracy_score(y_dev, svm_y_dev_predictions))

print("Dev Classification Report:")
print(classification_report(y_dev, svm_y_dev_predictions))

print("Dev Confusion Matrix:")
print(confusion_matrix(y_dev, svm_y_dev_predictions))

In [ ]:
svm_scores = svm_model.decision_function(x_dev_scaled)

from sklearn.metrics import roc_curve
import numpy as np

svm_fpr, svm_tpr, svm_thresholds = roc_curve( y_dev, svm_scores, pos_label=1)

svm_fnr = 1 - svm_tpr

svm_eer_idx = np.nanargmin(np.abs(svm_fnr - svm_fpr))
svm_eer = svm_fpr[svm_eer_idx]
svm_eer_threshold = svm_thresholds[svm_eer_idx]

print("Dev EER:", svm_eer)
print("EER Threshold:", svm_eer_threshold)

## Classifier Using Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=300, max_depth=None, min_samples_split=2, min_samples_leaf=1, class_weight="balanced",random_state=42)

rf_model.fit(x_train_scaled, y_train)
# Model has been trained.

In [ ]:
rf_y_dev_predictions = rf_model.predict(x_dev_scaled)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("RF Dev Accuracy:")
print(accuracy_score(y_dev, rf_y_dev_predictions))

print("RF Dev Classification Report:")
print(classification_report(y_dev, rf_y_dev_predictions))

print("RF Dev Confusion Matrix:")
print(confusion_matrix(y_dev, rf_y_dev_predictions))

In [ ]:
rf_y_dev_probability = rf_model.predict_proba(x_dev_scaled)[:, 1]

from sklearn.metrics import roc_curve
import numpy as np

rf_fpr, rf_tpr, rf_thresholds = roc_curve(y_dev, rf_y_dev_probability)
rf_fnr = 1 - rf_tpr

rf_eer_idx = np.nanargmin(np.abs(rf_fnr - rf_fpr))
rf_eer = rf_fpr[rf_eer_idx]
rf_eer_threshold = rf_thresholds[rf_eer_idx]

print("RF Dev EER:", rf_eer)
print("RF EER Threshold:", rf_eer_threshold)

## Comparing different classifier based on ROC Curves on Development Set

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 7))

plt.plot(lr_fpr, lr_tpr, label=f"Logistic Regression (EER={lr_eer:.4f})")
plt.plot(svm_fpr, svm_tpr, label=f"SVM (EER={svm_eer:.4f})")
plt.plot(rf_fpr, rf_tpr, label=f"Random Forest (EER={rf_eer:.4f})")

# Random baseline
plt.plot([0, 1], [0, 1], 'k--', label="Random")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves on Development Set")
plt.legend()
plt.grid(True)
plt.show()

## FInal Results and Model Finalising.

Performance was evaluated using:
* Accuracy
* Precision, Recall, F1-score
* Confusion Matrix
* Receiver Operating Characteristic (ROC) curve
* Equal Error Rate (EER)

Note: “Due to class imbalance, accuracy alone is insufficient; therefore, EER and class-wise recall are emphasized.”

| Model               | Accuracy    | EER        | Strengths              | Weaknesses                  |
| ------------------- | ----------- | ---------- | ---------------------- | --------------------------- |
| Logistic Regression | Medium      | High       | Simple, fast           | Poor non-linearity handling |
| Random Forest       | High        | Medium     | Strong spoof detection | Bias toward majority class  |
| **SVM**             | **Highest** | **Lowest** | Best generalization    | Higher computational cost   |


Final Model Selection

Based on:

* Highest accuracy
* Best ROC curve
* Lowest Equal Error Rate

*Support Vector Machine (SVM) was selected as the best classical machine learning model for spoof detection in this study.*